In [1]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [21]:
import pandas as pd
df = pd.DataFrame(db.comp.find({
    "offices":{
        "$not":{
            "$size":0
        }
    }},{"name":1,
       "offices":1,
       "category_code":1,
       "description":1,
       "products":1,
       "total_money_raised":1,
       "relationships":1,
       "competitions":1,
       "founded_year":1
      }
))
df.head()

,_id,category_code,competitions,description,founded_year,name,offices,products,relationships,total_money_raised
0,52cdef7c4bab8bd675297d8b,enterprise,[],Server Management Software,1996.0,AdventNet,"[{'description': 'Headquarters', 'address1': '...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",$0
1,52cdef7c4bab8bd675297d8a,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",Technology Platform Company,2005.0,Wetpaint,"[{'description': '', 'address1': '710 - 2nd Av...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...",$39.8M
2,52cdef7c4bab8bd675297d8c,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Online Business Apps Suite,2005.0,Zoho,"[{'description': 'Headquarters', 'address1': '...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...",$0
3,52cdef7c4bab8bd675297d8d,news,"[{'competitor': {'name': 'Reddit', 'permalink'...",user driven social content website,2004.0,Digg,"[{'description': None, 'address1': '135 Missis...","[{'name': 'Digg', 'permalink': 'digg'}]","[{'is_past': False, 'title': 'CEO', 'person': ...",$45M
4,52cdef7c4bab8bd675297d8f,network_hosting,"[{'competitor': {'name': 'Dropbox', 'permalink...",None,2005.0,Omnidrive,"[{'description': '', 'address1': 'Suite 200', ...","[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]","[{'is_past': True, 'title': 'Co-founder', 'per...",$800k


In [20]:

df.columns

Index(['_id', 'category_code', 'competitions', 'description', 'founded_year',
       'name', 'products', 'relationships', 'total_money_raised'],
      dtype='object')

In [23]:
df_copia=df[["offices","products","relationships","competitions"]].copy()
def cleaning(x):
    names=[]
    for e in x["products"]:
        names.append(e["name"])
        print(names)

df_copia.head().apply(cleaning,axis=1,result_type="expand")

['Wikison Wetpaint']
['Wikison Wetpaint', 'Wetpaint Social Distribution System']
['Zoho Office Suite']
['Zoho Office Suite', 'Zoho CRM']
['Zoho Office Suite', 'Zoho CRM', 'Zoho Assist']
['Digg']
['Omnidrive']


0    None
1    None
2    None
3    None
4    None
dtype: object